In [1]:
import numpy as np
import pandas as pd

datapath = "./data/Load_Current_Voltage_PF.xlsx"


## Use the real dataset and apply Constrained Optimisation for solving S_r, S_x

In [2]:
I_mag = pd.read_excel(datapath, sheet_name='Current(A)', header=None, index_col=None) 
V_mag = pd.read_excel(datapath, sheet_name='Voltage(V)', header=None, index_col=None) 
cos_ph = pd.read_excel(datapath, sheet_name='PF', header=None, index_col=None) # cos value of phase angle
node_num = cos_ph.shape[0]

In [3]:
cos_ph.shape


(9, 16384)

In [4]:
# tan_ph = np.sqrt(1-cos_ph**2)/cos_ph
sin_ph = np.sqrt(1-cos_ph**2)
I_real = I_mag * cos_ph
I_react = I_mag * sin_ph


# Compute the "differences" measure of Re(delta_I) --> (N x t)
delta_I_real = I_real.diff(axis=1)
delta_I_real.drop(columns=delta_I_real.columns[0], axis=1, inplace=True)
# Compute the "differences" measure of Im(delta_I) --> (N x t)
delta_I_react = I_react.diff(axis=1)
delta_I_react.drop(columns=delta_I_react.columns[0], axis=1, inplace=True)

# Compute the "differences" measure of Re(delta_V) --> (N x t)
delta_V = V_mag.diff(axis=1)
delta_V.drop(columns=delta_V.columns[0], axis=1, inplace=True)



# concatinate real and reactive I virtically into one matrix
delta_I = pd.concat([delta_I_real, delta_I_react], axis=0) # --> (2N x t)

print(delta_I.shape)
print(delta_V.shape)

(18, 16383)
(9, 16383)


solve the constrained opt problem (not consider the Distance matrix inequality constriant yet !)

In [5]:
import cvxpy as cp
# define symmetic matrix and variable in cvxpy, check: https://stackoverflow.com/questions/70257737/create-matrix-from-vector-cvxpy-variable

# assume 9 leaf nodes and only consider time step=1
n = node_num

S_r = cp.Variable((n,n), symmetric=True)
S_x = cp.Variable((n,n), symmetric=True)

constraints =[]

# constraint: S_r<=0 , S_x<=0 to all its elements in equ(3)
for row in range(n):
    for col in range(n):
        constraints +=[
            S_r[row][col] <=0
        ]
        
for row in range(n):
    for col in range(n):
        constraints +=[
            S_x[row][col] <=0
        ]
        
        
# constraint: S_(r,ii) <= S_(r,ij) and S_(x,ii) <= S_(x,ij) to all its elements in equ(4)
for i in range(n):
    constraints += [
        n*S_r[i][i] <= cp.sum(S_r[i][:]) # ex: 4*s1 <= s1 + s2 + s3 + s4
        ]
    
for i in range(n):
    constraints += [
        n*S_x[i][i] <= cp.sum(S_x[i][:]) # ex: 4*s1 <= s1 + s2 + s3 + s4
        ]



# Build the distance matrix D_r
diag_var_r = cp.diag(S_r) # extract the diagnal of S_r
S_rjj = cp.vstack([diag_var_r]*n)
S_rii = S_rjj.T
D_r = 2*S_r - S_rii - S_rjj

# Build the distance matrix D_x
diag_var_x = cp.diag(S_x) # extract the diagnal of S_r
S_xjj = cp.vstack([diag_var_x]*n)
S_xii = S_xjj.T
D_x = 2*S_x - S_xii - S_xjj


# Distance Matrix D_r >=0 to all its elements
for i in range(n):
    for j in range(n):
       constraints += [
           D_r[i][j] >=0
           ]

# Distance Matrix D_x >=0 to all its elements
for i in range(n):
    for j in range(n):
       constraints += [
           D_x[i][j] >=0
           ]
       


# Build the A_r matrix
A_r = np.eye(n) - (1/n) * np.ones((n,n)) @ D_r @ np.eye(n) - (1/n) * np.ones((n,n))

# Build the A_x matrix
A_x = np.eye(n) - (1/n) * np.ones((n,n)) @ D_x @ np.eye(n) - (1/n) * np.ones((n,n))

# constraint: 
constraints += [
    A_r <= 0,
    A_x <=0
]


## Distance constraint in equ(9) is (--> its not solvable if added)
# for i in range(n):
#     for j in range(n):
#         to_exclude = [i,j]
#         for k in range(n):
#             if k not in to_exclude:
#                 constraints += [
#                     D_r[i][j] <= D_r[i][k] + D_r[j][k]
#                 ]
# for i in range(n):
#     for j in range(n):
#         to_exclude = [i,j]
#         for k in range(n):
#             if k not in to_exclude:
#                 constraints += [
#                     D_x[i][j] <= D_x[i][k] + D_x[j][k]
#                 ]
                






S = cp.hstack([S_r, S_x])

objective = cp.Minimize(cp.sum_squares(S @ delta_I - delta_V))
prob = cp.Problem(objective, constraints)
prob.solve(verbose=True)

solution_S = S.value
solution_Sr, solution_Sx = out1, out2 = np.hsplit(solution_S, 2)


                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 15 01:56:43 PM: Your problem has 162 variables, 344 constraints, and 0 parameters.
(CVXPY) May 15 01:56:43 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 15 01:56:43 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 15 01:56:43 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 15 01:56:43 PM: Compiling problem (target solver=OSQP).
(CVXPY) May 15 01:56:43 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuff

In [6]:
print(f"Sr shape is:\n {S_r.shape}")
print(f"Sr is:\n {solution_Sr}")

Sr shape is:
 (9, 9)
Sr is:
 [[-4.48947039e-01 -1.19461884e-09 -1.49026746e-09 -9.73829655e-02
  -2.10655473e-09 -1.43118656e-09 -1.42292903e-01 -5.89871262e-10
  -1.34254298e-09]
 [-1.19461884e-09 -9.88766631e-01 -5.80278172e-10  2.67164314e-09
  -1.35739387e-01  9.49635039e-11 -1.32204904e-10 -1.05667823e-01
   1.83004356e-11]
 [-1.49026746e-09 -5.80278172e-10 -4.23568616e-01  2.89683734e-09
  -1.06683676e-09 -1.00940795e-01 -8.12318803e-10  4.77907592e-11
  -4.99106137e-02]
 [-9.73829655e-02  2.67164314e-09  2.89683734e-09 -4.84208259e-01
   3.00961280e-09  3.05307502e-09 -2.65707282e-01  6.44504591e-09
   2.68416707e-09]
 [-2.10655473e-09 -1.35739387e-01 -1.06683676e-09  3.00961280e-09
  -4.96754804e-01 -4.99217061e-10 -9.07209509e-10 -2.71263673e-01
  -4.73219922e-10]
 [-1.43118656e-09  9.49635039e-11 -1.00940795e-01  3.05307502e-09
  -4.99217061e-10 -6.03743279e-01 -2.09943247e-10  3.73323583e-10
  -2.98163725e-01]
 [-1.42292903e-01 -1.32204904e-10 -8.12318803e-10 -2.65707282e-01

In [7]:
print(f"Sx shape is:\n {S_x.shape}")
print(f"Sx is:\n {solution_Sx}")

Sx shape is:
 (9, 9)
Sx is:
 [[-3.55442766e-01 -1.06027578e-09 -1.78875632e-09  5.80748401e-13
  -2.21610112e-09 -1.29179324e-09 -2.23065809e-02 -8.73500190e-10
  -1.19419593e-09]
 [-1.06027578e-09 -1.59141841e+00 -7.44856321e-10  3.42061123e-09
  -8.89120411e-12  8.95039112e-11 -1.67498740e-10 -2.12810064e-02
   1.41460022e-11]
 [-1.78875632e-09 -7.44856321e-10 -4.01181036e-01  3.72481129e-09
  -1.38842629e-09 -2.99032229e-02 -1.04182320e-09  6.81447693e-11
  -1.52487311e-01]
 [ 5.80748401e-13  3.42061123e-09  3.72481129e-09 -3.68531825e-01
   3.87920255e-09  3.93271934e-09 -6.81183786e-02  8.91382995e-09
   3.44255708e-09]
 [-2.21610112e-09 -8.89120411e-12 -1.38842629e-09  3.87920255e-09
  -3.63136841e-01 -5.78893655e-10 -1.08324853e-09 -4.92358512e-02
  -5.26658546e-10]
 [-1.29179324e-09  8.95039112e-11 -2.99032229e-02  3.93271934e-09
  -5.78893655e-10 -3.85033052e-01 -2.51436684e-10  5.40775593e-10
  -9.59693793e-02]
 [-2.23065809e-02 -1.67498740e-10 -1.04182320e-09 -6.81183786e-02

In [8]:
solution_Sr - solution_Sr.T # check symmetric of S_r

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
solution_Sx - solution_Sx.T # check symmetric of S_x

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [13]:
# save distance matrix for step 2 graph reconstruction
np.save('D_r.npy', D_r.value)
np.save('D_x.npy', D_x.value)

## Recursize Grouping with BackTracking (Section B)

- testing...
- Graph recursive find shortest path between two nodes:  https://www.python.org/doc/essays/graphs/
- check this "Chou-Liu Algorithm" for treee topology reconstruction: https://pgmpy.org/examples/Structure%20Learning%20with%20Chow-Liu.html

or other implementatino fo chou-Liu Alg at: https://github.com/JohnReid/pybool/blob/master/python/pybool/chow_liu_trees.py

or chou-Liu tree at : https://pgmpy.org/structure_estimator/tree.html


- Constructing a tree based on a given distance matrix: https://www.kuniga.me/blog/2019/05/10/constructing-trees-from-a-distance-matrix.html

- Step by step code building tree from given distance matrix: https://www.tenderisthebyte.com/blog/2022/08/31/neighbor-joining-trees/






testing below....

In [19]:
# # from https://pgmpy.org/structure_estimator/tree.html

# import numpy as np
# import pandas as pd
# import networkx as nx
# import matplotlib.pyplot as plt
# from pgmpy.estimators import TreeSearch
# values = pd.DataFrame(np.random.randint(low=0, high=2, size=(1000, 5)),
#                       columns=['A', 'B', 'C', 'D', 'E'])
# # est = TreeSearch(values, root_node='B')
# # model = est.estimate(estimator_type='chow-liu')
# # nx.draw_circular(model, with_labels=True, arrowsize=20, arrowstyle='fancy',
# #                  alpha=0.3)
# # plt.show()
# est = TreeSearch(values)
# model = est.estimate(estimator_type='chow-liu')
# nx.draw_circular(model, with_labels=True, arrowsize=20, arrowstyle='fancy',
#                  alpha=0.3)
# plt.show()


- Neighbor joining tree based on distance matrix referred from https://www.tenderisthebyte.com/blog/2022/08/31/neighbor-joining-trees/